In [ ]:
# Markdown 
Otras formas de trabajo
#   Datos a usar
cuántitativas: 
P3094S3 : Cuánto ahorro por cultivar
P3087S1: Valor mensual por practicas o pasantías
P3095S3: Valor ahorra por criar animales
# Cualitativas
P3101: ¿Fue a reuniones familiares durante las ultimas 4 semanas? si o no

# agrupar estas variables para los 3 meses 